# 🎬 Video Data Analysis Roadmap

*This notebook provides a structured roadmap for analyzing video datasets. It guides you step-by-step through setup, loading, cleaning, temporal/spatial preprocessing, exploratory analysis, augmentation, and evaluation. Students can follow these stages to prepare and analyze video data for tasks such as classification, action recognition, detection, tracking, segmentation, and captioning.*

---

## 1) Setup & Imports

**Core**
* `numpy`, `pandas`, `matplotlib.pyplot`, `pathlib`, `tqdm`, `json`, `yaml`
* Video IO/decoding: `opencv-python` (`cv2`), `PyAV`, `decord`, `ffmpeg`/`ffmpeg-python`
* Frames & images: `PIL.Image`

**DL stacks (choose one)**
* PyTorch (+ `torchvision`, `pytorchvideo`) or TensorFlow/Keras (`tf.data`, `tf.io`)
* Augmentations: `albumentations` (frame-wise), `torchvision.transforms`, or temporal aug libs

**Audio (optional)**
* `librosa`, `torchaudio` for audio tracks

**Reproducibility**
* Fix seeds (`random`, `numpy`, framework)
* Log package versions (`pip freeze`), GPU & decoder info (CUDA/cuDNN/NVDEC availability)

---

## 2) Load Video Data

**Folder structures**
* Classification: `videos/{train,val,test}/{class_name}/*.mp4`
* Detection/Segmentation/Tracking: `videos/*.mp4` + annotations (e.g., AVA/COCO-VID/YouTube‑VIS, MOT, custom JSON with per‑frame boxes/masks/tracks)
* Captioning: `videos/*.mp4` + `captions.json` (video_id → text)
* External sources: Kaggle ZIPs, S3/Drive buckets, long-form originals to be clipped

**Methods**
* Build a **manifest DataFrame**: `video_path, split, label(s)/task, duration_sec, fps, width, height, codec, audio_present, checksum/hash`
* **Probe metadata** (ffprobe/pyav): fps, resolution, duration, bit‑rate, codec, audio streams
* For detection/seg/track: **parse annotation files**; verify frame indices/timecodes align with decoded frames
* If using *clips* (fixed length): precompute start times (#clips per video) for reproducible sampling

**Task-specific Data Loaders (must build; no code here)**
* **Video classification** → returns `(clip_tensor, class_label)` for fixed‑length clips
* **Action localization/detection** → `(clip_tensor, temporal_segments and/or spatiotemporal boxes, labels)`
* **Object tracking / MOT** → `(frames_sequence, tracks per frame)`
* **Video segmentation** → `(frames_sequence, masks_sequence)` with matched sizes
* **Video captioning** → `(clip_tensor, caption_text)`

**Loader requirements**
* Consistent **clip sampling policy** (uniform, random, center, multi‑crop; #frames, stride)
* Apply **spatial** (resize/normalize) and **temporal** (frame rate, stride) preprocessing
* Ensure alignment between frames and labels (frame indices vs timestamps)
* Clean support for `train/val/test` splits; modular and swappable

**Quick checks**
* Can each video be opened/decoded to frames?
* Any **zero‑duration**, **0‑byte**, or **mismatched fps** files?
* Do decoded frame counts match metadata expectation (within tolerance)?

---

## 3) Inspect Video Data (Initial Profiling)

**Video metadata**
* Distributions of duration, fps, width/height, aspect ratios, codecs, bitrates
* Audio track presence/length (if using audio‑visual models)

**Dataset composition**
* Class counts and imbalance (classification)
* For detection/segmentation: avg boxes/masks per frame, object category frequency
* For tracking: avg track length, #IDs per video, ID switches

**Visual peeks**
* Contact sheets / frame grids sampled from random clips per class
* Short GIFs of a few examples per class/task
* Overlays of a few annotations (boxes/masks/tracks) to spot obvious issues

---

## 4) Data Cleaning (Quality & Integrity)

**File & stream integrity**
* Remove corrupted or truncated videos; transcode problematic codecs/containers if necessary
* Fix incorrect fps metadata (variable frame rate → consider re‑encoding to CFR) 
* Deinterlace if interlaced sources

**Duplicates & near‑duplicates**
* Perceptual **frame hashing** on keyframes (every N frames) and cluster by Hamming distance
* Ensure no near‑duplicates **cross splits** (train/val/test leakage)

**Label QA**
* Temporal alignment: are action segments within duration? 
* Spatiotemporal boxes inside frame bounds; masks match frame sizes
* Track continuity: check for impossible jumps; ID consistency across frames
* Class ontology normalization

**Quality issues**
* Extreme blur, noise, heavy compression blocks, dropped frames
* Sudden fps changes, A/V desync
* Camera artifacts (rolling shutter, flicker)

**Ethics/PII**
* Faces/plates policy; consent requirements; copyright and source logs

---

## 5) Splitting Strategy (Prevent Leakage)

**Methods**
* **Stratified** split by label for classification
* **Group‑aware** split by source (movie, episode, camera/site, subject/patient, event, YouTube channel)
* **Time‑aware** split for chronological generalization
* Maintain similar distributions of duration/fps/resolution across splits

**Validation design**
* K‑fold or GroupKFold for small datasets
* Fixed hold‑out + cross‑val when tuning

---

## 6) Preprocessing (Spatial & Temporal)

**Temporal sampling**
* Fixed clip length (e.g., 16/32/64 frames) with **stride** S
* Policies: center, uniform, random, multi‑clip per video; warm‑up frames for optical flow
* Convert VFR → CFR or resample frames to a **target fps** for consistency

**Spatial processing**
* Resize to a canonical short‑side (e.g., 256) + center/letterbox crop to model input (e.g., 224×224)
* Normalize pixel values; choose color space consistently (BGR↔RGB)

**Audio (if used)**
* Resample to target sample rate; compute mel‑spectrograms/log‑mels as inputs

**Caching**
* Cache decoded frames/clips or use memory‑mapped datasets to avoid re‑decoding

---

## 7) Data Augmentation (Spatial + Temporal)

**Spatial**
* Horizontal flips, small rotations, random resized crops, affine, color jitter, grayscale, gaussian noise
* For detection/segmentation: box/mask‑aware transforms

**Temporal**
* Random frame‑drop, temporal jitter, time warping (mild), clip reversal (if label‑invariant)
* Multi‑clip sampling per epoch for robustness

**Compression/robustness**
* JPEG/MPEG artifact simulation, bitrate jitter, Gaussian blur

**Ablations**
* With/without augmentations; spatial‑only vs temporal‑only; impact on each metric

---

## 8) Feature Engineering (Optional)

**Classical**
* Frame‑level color/texture histograms aggregated over time
* Optical flow / motion histograms (HOF), HOG3D, dense trajectories (if feasible)

**Deep features**
* Frame embeddings from 2D CNN (ResNet/ViT) + temporal pooling (mean, NetVLAD)
* 3D CNN/TimeSformer/SlowFast features
* Audio embeddings (VGGish/YAMNet/AST) and early/late fusion

---

## 9) Exploratory Data Analysis (EDA)

**Distributions**
* Labels per class; clip duration; fps; resolution; bitrate
* #instances per frame/time for detection/seg; track length distributions for MOT

**Qualitative panels**
* Before/after preprocessing and augmentations
* Outlier gallery: very short/long clips, odd fps, corrupted frames

**Embeddings**
* t‑SNE/UMAP on clip‑level embeddings for cluster separability & mislabels

**Leakage & imbalance**
* Near‑duplicate matrices across splits (via keyframe hashes)
* Class imbalance visuals → decide on sampling/weights/focal loss

---

## 10) Baselines &  Checks

**Baselines**
* Frame‑sample baseline: classify using a single center frame (upper bound for non‑temporal)
* 2D CNN + temporal pooling vs. simple 3D CNN (e.g., R(2+1)D) small model

**Sanity**
* Label shuffle test (should fail)
* Overfit a tiny subset (should succeed)
* Quick Grad‑CAM/attention rollout on a few clips to verify focus

---

## 11) Documentation & Dataset Card

**Include**
* Sources, licenses, collection periods, consent/PII notes
* Preprocessing & temporal sampling policies
* Known limitations/biases; camera/sensor diversity
* Split rationale & leakage checks
* Versioning & changelog; exact commands to rebuild

---

## 12) Packaging for Training

**Pipelines**
* PyTorch `Dataset`/`DataLoader` for video clips or TensorFlow `tf.data`
* Decoders: `decord`, `pyav`, `opencv`; prefer zero‑copy GPU decode if available (NVDEC, torchvision.io)
* Convert to sharded formats: **WebDataset** tar shards, **TFRecord**, or frame folders with manifests

**Performance**
* Pre‑decode/frame caches; persistent workers; pinned memory; prefetch
* Mixed precision & gradient checkpointing readiness
* Avoid Python GIL hotspots by using compiled decoders (decord/pyav) and multi‑processing

---

## 13) Readiness Checklist (Go/No-Go)

* [ ] All videos decode; metadata sane (fps, duration, resolution)
* [ ] No cross‑split near‑duplicates or same‑scene leakage
* [ ] Labels valid & aligned (temporal/spatial); tracks consistent
* [ ] Splits balanced; distributions comparable
* [ ] Preprocessing & augmentation (spatial+temporal) defined & reproducible
* [ ] EDA completed; issues addressed or documented
* [ ] Baseline sanity checks passed
* [ ] Dataset card complete; licenses/consent verified

---

## 14) Task‑Specific Metrics (for when you evaluate)

* **Video classification**: Top‑1/Top‑5 accuracy, macro F1, AUROC (multi‑label: mAP)
* **Action detection/localization**: mAP at temporal IoU thresholds; spatiotemporal mAP
* **Video object detection**: mAP@[.5:.95] on video benchmarks
* **Tracking (MOT)**: MOTA/MOTP/HOTA, IDF1, ID switches
* **Video segmentation**: Video mIoU, region F‑score, boundary metrics
* **Captioning**: BLEU, METEOR, ROUGE‑L, CIDEr, SPICE
* **Robustness**: performance under fps jitter, compression, occlusion

---

### Notes for Students

* **Start with clear clip sampling** (length, stride, fps) and keep it consistent.
* **Measure effect** of temporal augmentations separately from spatial ones.
* **Document decoding choices** (decoder backend, CFR/VFR handling) — they affect reproducibility and speed.
